In [60]:
gene_expr_bed = "/Users/sabrinami/Library/CloudStorage/Box-Box/imlab-data/data-Github/rat-genomic-analysis/Enformer-Predictions/data/Brain.rn7.expr.tpm.bed"

import pandas as pd
import numpy as np
obs_gene_expr = pd.read_csv(gene_expr_bed, sep="\t", header=0, index_col='gene_id')

/var/folders/1q/nqyt2x9s071fkn235hfq1p680000gn/T/ipykernel_29757/2078132498.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  obs_gene_expr = pd.read_csv(gene_expr_bed, sep="\t", header=0, index_col='gene_id')


In [4]:

obs_gene_expr.head()

,#chr,start,end,000789972A,0007899884,000789FF64,00079298B1,00079299B5,0007929A79,0007929B36,...,0007899761,0007899866,00078A00E6,0007929938,0007929956,00079FF7D6,0007899826,0007929917,00077E82E3,00078A18A7
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSRNOG00000065394,1,31563,31564,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
ENSRNOG00000065356,1,41634,41635,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
ENSRNOG00000070568,1,76833,76834,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0
ENSRNOG00000065706,1,184235,184236,0.0,0.0,0.0,0.0,0.18,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
ENSRNOG00000069644,1,272252,272253,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


In [98]:
gene_list = ['ENSRNOG00000001342', 'ENSRNOG00000020624']
gene = gene_list[0]
#obs_gene_expr[obs_gene_expr["gene_id"].isin(gene_list)]

pred_vs_obs = {}
for gene in gene_list:
    obs = obs_gene_expr.loc[gene][3:]
    print(obs)
    pred_vs_obs[gene] = pd.DataFrame({"observed": obs, "predicted": np.zeros(obs.shape)})

000789972A     7.53
0007899884    11.35
000789FF64     9.33
00079298B1     9.88
00079299B5      9.9
              ...  
00079FF7D6     7.63
0007899826     2.83
0007929917     8.29
00077E82E3    10.05
00078A18A7     6.73
Name: ENSRNOG00000001342, Length: 340, dtype: object
000789972A    44.43
0007899884    50.12
000789FF64    62.47
00079298B1    64.97
00079299B5    57.53
              ...  
00079FF7D6     53.3
0007899826    52.88
0007929917    54.19
00077E82E3    74.91
00078A18A7     50.0
Name: ENSRNOG00000020624, Length: 340, dtype: object


In [51]:
def compute_tss_bin(output_window_coords_resized, tss_position, output_length=114688, bin_length=128, rc=False):
    '''
    Computes the bin index of the enformer output bin containing the TSS
    Inputs:
    output_window_coords_resized: Kipoiseq interval for the output window of Enformer, should already be resized to the model output window size
    tss_position: absolute location of TSS
    output_length: Length of output window in base pairs
    bin_size: Length of output bins in base pairs
    Outputs:
    tss_bin: Index of bin containing TSS
    '''
    try:
        relative_position = int(tss_position) - int(output_window_coords_resized.start)
    except:
    # print(tss_position)
    # print(output_window_coords_resized.start)
        sys.exit()

    if relative_position < 0 or relative_position > output_length:
        # print(tss_position, "appears to be out of bounds for output window: ",output_window_coords_resized)
        return None 
    else:
        center = relative_position // bin_length

        
        if rc:
            number_of_bins = int(output_length/bin_length)
            new_center = number_of_bins - 1 - center
            # print(new_center)
            return (new_center-1,new_center,new_center+1)
        else:
            # print(center)
            return (center-1,center,center+1)

In [20]:
annot_df = pd.read_csv("/Users/sabrinami/Library/CloudStorage/Box-Box/imlab-data/data-Github/rat-genomic-analysis/Enformer-Predictions/data/rn7.gene.txt", sep="\t", index_col='geneId')
annot_df.head()

/var/folders/1q/nqyt2x9s071fkn235hfq1p680000gn/T/ipykernel_29757/10523050.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  annot_df = pd.read_csv("/Users/sabrinami/Library/CloudStorage/Box-Box/imlab-data/data-Github/rat-genomic-analysis/Enformer-Predictions/data/rn7.gene.txt", sep="\t", index_col='geneId')


,geneSymbol,chromosome,start,end,strand,tss,description,hasEqtl,expr_BLA,expr_Brain,...,sqtl_BLA,sqtl_Brain,sqtl_Eye,sqtl_IL,sqtl_LHb,sqtl_NAcc,sqtl_NAcc2,sqtl_OFC,sqtl_PL,sqtl_PL2
geneId,,,,,,,,,,,,,,,,,,,,,
ENSRNOG00000066169,ENSRNOG00000066169,1,36112690,36122387,-,36122387,NaN,True,True,True,...,False,False,False,False,False,False,False,False,False,False
ENSRNOG00000070168,Olr56,1,157231467,157232417,+,157231467,olfactory receptor family 51 subfamily F membe...,True,False,True,...,False,False,False,False,False,False,False,False,False,False
ENSRNOG00000070901,Irgq,1,80123925,80131881,+,80123925,immunity-related GTPase Q,True,True,True,...,False,False,False,False,False,False,False,False,False,False
ENSRNOG00000018029,Doc2g,1,201296342,201299865,+,201296342,"double C2-like domains, gamma",False,True,True,...,False,False,False,False,False,False,False,False,False,False
ENSRNOG00000031391,Ceacam16,1,79514975,79524871,-,79524871,"CEA cell adhesion molecule 16, tectorial membr...",True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [108]:
import kipoiseq # for manipulating fasta files
from kipoiseq import Interval # same as above, really
import sys
import h5py
predictions_dir = "/Users/sabrinami/Library/CloudStorage/Box-Box/imlab-data/data-Github/rat-genomic-analysis/Enformer-Predictions/predictions"

for gene in gene_list:
    gene_annot = annot_df.loc[gene]
    interval_obj = Interval("chr" + str(gene_annot['chromosome']), gene_annot['start'], gene_annot['end'])
    tss_bins = compute_tss_bin(interval_obj.resize(114688), gene_annot['tss'])
    interval = f"chr{gene_annot['chromosome']}_{gene_annot['start']}_{gene_annot['end']}"
    for individual in pred_vs_obs[gene].index:
        individual = "00079FB742"
        predictions = h5py.File(f"{predictions_dir}/{individual}/haplotype1/{interval}_predictions.h5", "r")
        matrix = predictions[interval][()]
        pred_expr = np.average(matrix[tss_bins, 4980])
        pred_vs_obs[gene].loc[individual, "predicted"] = pred_expr
        break

           observed  predicted
000789972A     7.53        0.0
0007899884    11.35        0.0
000789FF64     9.33        0.0
00079298B1     9.88        0.0
00079299B5      9.9        0.0
...             ...        ...
00079FF7D6     7.63        0.0
0007899826     2.83        0.0
0007929917     8.29        0.0
00077E82E3    10.05        0.0
00078A18A7     6.73        0.0

[340 rows x 2 columns]
           observed  predicted
000789972A    44.43        0.0
0007899884    50.12        0.0
000789FF64    62.47        0.0
00079298B1    64.97        0.0
00079299B5    57.53        0.0
...             ...        ...
00079FF7D6     53.3        0.0
0007899826    52.88        0.0
0007929917    54.19        0.0
00077E82E3    74.91        0.0
00078A18A7     50.0        0.0

[340 rows x 2 columns]
